# Create an example fUS-BIDS v0.0.5 dataset

This notebook creates an example fUS-BIDS dataset for version 0.0.5 of the
specification. The data comes from the Opioids project, conducted by Jean-Charles
Mariani under the supervision of Andrea Kliewer and Zsolt Lenkei. 

In [16]:
import json
import random

import numpy as np
import pandas as pd
from pathlib import Path

import pythmed

rawdata_path = Path(
    "/home/sdiebolt/Documents/Work/fus-bids-example-dataset/"
    "fusbids-v0.0.5-example/rawdata"
)
rawdata_path.mkdir(parents=True, exist_ok=True)

## Dataset description

The first step is to create the `dataset_description.json` file.

In [63]:
dataset_description_path = rawdata_path / "dataset_description.json"
dataset_description = {
    "Name": "fUS-BIDS v0.0.5 example dataset",
    "BIDSVersion": "1.8.0",
    "DatasetType": "raw",
    "Authors": ["Samuel Diebolt"],
    "GeneratedBy": [
        {
            "Name": "Custom script",
            "Description": "Custom Python notebook used to convert machine output to BIDS.",
        }
    ],
}

with open(dataset_description_path, "w") as dataset_description_file:
    dataset_description_file.write(json.dumps(dataset_description, indent=2))

## `README`

The `README` file should describe the dataset in more details.

In [64]:
readme_path = rawdata_path / "README.md"
readme = """# Example fUS-BIDS v0.0.5 dataset

This dataset provides an example for version 0.0.5 of the [fUS-BIDS
specification](https://docs.google.com/document/d/1W3z01mf1E8cfg_OY7ZGqeUeOKv659jCHQBXavtmT-T8/edit?pli=1#heading=h.4k1noo90gelw).

## Overwiew

This dataset consists of ten subject with two sessions each, vehicle and treatment. Each
session consists of a 3D angiography and three 2D+t functional scans.

Scans are represented by empty NIfTI files: the purpose of this dataset is to
demonstrate JSON and TSV metadata files.

## Authors

- [Samuel Diebolt](mailto:samuel@diebolt.io)"""

with open(readme_path, "w") as readme_file:
    readme_file.write(readme)

## `CHANGES`

The `CHANGES` file should describe the different versions of the dataset. It must follow
the [CPAN Changelog convention](https://metacpan.org/pod/release/HAARG/CPAN-Changes-0.400002/lib/CPAN/Changes/Spec.pod).

In [65]:
changes_path = rawdata_path / "CHANGES.md"
changes = """1.0.0 2023-06-21
  
 - Initial release."""

with open(changes_path, "w") as changes_file:
    changes_file.write(changes)

## Participants

The `participants.tsv` and `participants.json` files should describe properties of
participants such as age, sex, strain, etc.

In [66]:
participants_table_path = rawdata_path / "participants.tsv"
participants_table = {
    "participant_id": [f"sub-{i:02d}" for i in range(1, 11)],
    "species": ["mus musculus"] * 10,
    "strain": ["C57BL/6J"] * 10,
    "strain_rrid": ["RRID:IMSR_JAX:000664"] * 10,
    "age": np.random.randint(25, 30, size=(10,)).tolist(),
    "sex": random.choices(["F", "M"], k=10),
    "weight": 2 * np.random.randn(10),
}
participants_table = pd.DataFrame(participants_table)


# Set weight based on sex.
participants_table.loc[participants_table.sex == "F", "weight"] += 26
participants_table.loc[participants_table.sex == "M", "weight"] += 35

participants_table.to_csv(participants_table_path, sep="\t")

In [67]:
participants_json_path = rawdata_path / "participants.json"
participants_json = {
    "species": {
        "Description": "binomial species name from NCBI Taxonomy (https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi)"
    },
    "strain": {"Description": "string value indicating the strain of the species"},
    "strain_rrid": {
        "Description": "research resource identifier of the strain of the species from https://scicrunch.org/resources/Organisms/search"
    },
    "age": {"Description": "Age of the animal at first acquisition, in weeks."},
    "sex": {"Description": "Sex of the animal", "Levels": {"M": "male", "F": "female"}},
    "weight": {"Description": "Weight of the animal at first acquisition, in grams."},
}

with open(participants_json_path, "w") as participants_json_file:
    participants_json_file.write(json.dumps(participants_json, indent=2))

## Commong `pwd` JSON file

All power Doppler scans have common metadata that must be specified at the root of the
dataset in a `pwd.json` file according to the BIDS inheritance principle.

In [15]:
pwd_json_path = rawdata_path / "pwd.json"
pwd_json = {
    "Manufacturer": "Iconeus",
    "ManufacturersModelName": "Iconeus One",
    "DeviceSerialNumber": "X23HFB12K8",
    "StationName": "Machine01",
    "SoftwareVersions": "1.5.0",
    "ProbeType": "linear",
    "ProbeModel": "IcoPrime",
    "ProbeCentralFrequency": 15.625,
    "ProbeNumberOfElements": 128,
    "ProbePitch": 0.11,
    "ProbeRadiusOfCurvature": 0,
    "ProbeElevationAperture": 1.5,
    "ProbeElevationFocus": 8,
    "MaximalDepth": 10,
    "UltrasoundPulseRepetitionFrequency": 5500,
    "PlaneWaveAngles": [-10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10],
    "UltrafastSamplingFrequency": 500,
    "ProbeVoltage": 25,
    "SequenceName": "default",
    "ClutterFilterWindowDuration": 400,
    "ClutterFilters": [{"FilterType": "Fixed-threshold SVD", "Threshold": 60}],
    "PowerDopplerIntegrationDuration": 400,
    "InstitutionName": "Physics for Medicine",
    "InstitutionAddress": "2 - 10 Rue d'Oradour-sur-Glane, 75015 Paris, France",
}

with open(pwd_json_path, "w") as pwd_json_file:
    pwd_json_file.write(json.dumps(pwd_json, indent=2))

## Common `task-rest` JSON file

All functional scans have common metadata that must be specified at the root of the
dataset in a `task-rest_pwd.json` file according to the BIDS inheritance principle.

In [ ]:
task_json_path = rawdata_path / "task-rest_pwd.json"
task_json = {
    "TaskName": "awake",
    "TaskDescription": "Awake head-fixed state",
    "VolumeTiming": np.arange(0, 1200, 0.4),
}

with open(task_json, "w") as task_json_file:
    task_json_file.write(json.dumps(task_json, indent=2))

## Individual scans

In [62]:
standard_sidecar_json = {
    "Manufacturer": "Iconeus",
    "ManufacturersModelName": "Iconeus One",
    "DeviceSerialNumber": "X23HFB12K8",
    "StationName": "Machine01",
    "SoftwareVersions": "1.5.0",
    "ProbeType": "linear",
    "ProbeModel": "IcoPrime",
    "ProbeCentralFrequency": 15.625,
    "ProbeNumberOfElements": 128,
    "ProbePitch": 0.11,
    "ProbeRadiusOfCurvature": 0,
    "ProbeElevationAperture": 1.5,
    "ProbeElevationFocus": 8,
    "MaximalDepth": 10,
    "UltrasoundPulseRepetitionFrequency": 5500,
    "PlaneWaveAngles": [-10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10],
    "UltrafastSamplingFrequency": 500,
    "ProbeVoltage": 25,
    "SequenceName": "default",
    "ClutterFilterWindowDuration": 400,
    "ClutterFilters": [{"FilterType": "Fixed-threshold SVD", "Threshold": 60}],
    "PowerDopplerIntegrationDuration": 400,
}

sessions = ("vehicle", "treatment")
acq_values = ("bregmaMinus2", "bregmaMinus1", "bregmaPlus05")
for subject_index in range(1, 11):
    subject_path = rawdata_path / f"sub-{subject_index:02d}"

    # Subject-wide power Doppler JSON file.
    subject_json_path = f"sub-{subject_index:02d}_pwd.json"
    with open(subject_json_path, "w") as subject_json_file:
        subject_json_file.write(json.dumps(standard_sidecar_json, indent=2))

    session_dates = []
    for session in sessions:
        scan_paths_in_session = []

        session_path = subject_path / f"ses-{session}"

        # Anatomical scans.
        anat_path = session_path / "anat"
        anat_path.mkdir(exist_ok=True, parents=True)

        angio_scan_path = (
            anat_path / f"sub-{subject_index:02d}_ses-{session}_pwd.nii.gz"
        )
        open(angio_scan_path, "a").close()
        scan_paths_in_session.append(str(angio_scan_path.relative_to(session_path)))

        # Functional scans.
        fus_path = session_path / "fus"
        fus_path.mkdir(exist_ok=True, parents=True)

        for acq_value in acq_values:
            fus_scan_path = fus_path / (
                f"sub-{subject_index:02d}_ses-{session}_task-awake"
                f"_acq-{acq_value}_pwd.nii.gz"
            )
            open(fus_scan_path, "a").close()
            scan_paths_in_session.append(str(fus_scan_path.relative_to(session_path)))

        # Task-wide JSON file.
        fus_sidecar_json_path = (
            fus_path / f"sub-{subject_index:02d}_ses-{session}_task-awake_pwd.json"
        )
        fus_sidecar_json = {
            "TaskName": "awake",
            "TaskDescription": "Awake head-fixed state",
            "VolumeTiming": np.arange(0, 1200, 0.4).tolist(),
        }
        with open(fus_sidecar_json_path, "w") as fus_sidecar_json_file:
            fus_sidecar_json_file.write(json.dumps(fus_sidecar_json, indent=2))

        # Scans TSV file.
        scans_table_path = (
            session_path / f"sub-{subject_index:02d}_ses-{session}_scans.tsv"
        )
        scan_dates = np.array(
            [
                datetime.datetime(2022, 1, 1)
                + datetime.timedelta(
                    days=random.randint(0, 360),
                    seconds=random.randint(0, 60),
                    minutes=random.randint(0, 60),
                    hours=random.randint(0, 24),
                )
                for _ in range(len(scan_paths_in_session))
            ]
        )
        session_dates.append(scan_dates.min())
        scans_table = pd.DataFrame(
            {
                "filename": scan_paths_in_session,
                "acq_time": [
                    date.strftime("%Y-%m-%dT:%H:%M:%S") for date in scan_dates
                ],
            }
        )
        scans_table.to_csv(scans_table_path, sep="\t")
    
    session_table_path = subject_path / f"sub-{subject_index:02d}_sessions.tsv"
    session_table = pd.DataFrame(
        {
            "session_id": [f"ses-{session}" for session in sessions],
            "acq_time": [
                date.strftime("%Y-%m-%dT:%H:%M:%S") for date in session_dates
            ]
        }
    )
    session_table.to_csv(session_table_path, sep="\t")

In [54]:
dates = [
            datetime.datetime(2022, 1, 1)
            + datetime.timedelta(
                days=random.randint(0, 360),
                seconds=random.randint(0, 60),
                minutes=random.randint(0, 60),
                hours=random.randint(0, 24),
            )
            for _ in range(10)
        ]

In [60]:
scans_table

,filename,acq_time
0,anat/sub-10_ses-treatment_pwd.nii.gz,2022-07-07T:23:47:16
1,fus/sub-10_ses-treatment_task-awake_acq-bregma...,2022-05-28T:03:26:25
2,fus/sub-10_ses-treatment_task-awake_acq-bregma...,2022-12-16T:05:36:53
3,fus/sub-10_ses-treatment_task-awake_acq-bregma...,2022-06-30T:01:12:38
